# Installations & Util

## Initialize Notebook

In [22]:
import ipywidgets as widgets
from IPython.display import Javascript

outputInit = widgets.Output()

btnInit = widgets.Button(
    description='Initialize Notebook',
    disabled=False,
    button_style='info',
)

def runInit(state):
    display(Javascript('IPython.notebook.execute_cells_below()'))
    with outputInit:
        print("Running notebook initialization..")

# btnInit.observe(runInit)
btnInit.on_click(runInit)
# ========== END TOGGLE BTN END ==========
display(btnInit, outputInit)

<IPython.core.display.Javascript object>

Button(button_style='info', description='Initialize Notebook', style=ButtonStyle())

Output()

## Install Requirements

In [42]:
!pip install -r requirements.txt --user

In [24]:
!pip install torch torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/cu117

Looking in indexes: https://pypi.org/simple, https://download.pytorch.org/whl/cu117


## Code Hiding

In [25]:
import ipywidgets as widgets
from IPython.display import display, HTML
# ========== TOGGLE BTN ==========
javascript_functions = {False: "hide()", True: "show()"}
button_descriptions  = {False: "Show code", True: "Hide code"}

def toggle_code(state):
    """
    Toggles the JavaScript show()/hide() function on the div.input element.
    """
    output_string = "<script>$(\"div.input\").{}</script>"
    output_args   = (javascript_functions[state],)
    output        = output_string.format(*output_args)
    display(HTML(output))

def button_action(value):
    """
    Calls the toggle_code function and updates the button description.
    """
    state = value.new
    toggle_code(state)
    value.owner.description = button_descriptions[state]
    
state = False
toggle_code(state)

button = widgets.ToggleButton(state, description = button_descriptions[state])
button.observe(button_action, "value")
# ========== END TOGGLE BTN END ==========
display(button)

ToggleButton(value=False, description='Show code')

# TSU Pipeline
This section in the notebook is delicated for the TSU pipline.

## Data Exploration
This section in the notebook allows the user to load and display video data from the Toyota Smarthome (TSU) project (https://project.inria.fr/toyotasmarthome/).

### File Upload

In [67]:
import ipywidgets as widgets
from ipywidgets import Button
from IPython.display import display, HTML
from ipyfilechooser import FileChooser
import zipfile as zf
import shutil

# https://thispointer.com/python-how-to-unzip-a-file-extract-single-multiple-or-all-files-from-a-zip-archive/#:~:text=Extract%20all%20files%20from%20a%20zip%20file%20to%20different%20directory,can%20be%20relative%20or%20absolute.&text=It%20will%20extract%20all%20the%20files%20in%20%27sample,zip%27%20in%20temp%20folder.

# ===================== FileChooser =====================
# Create and display a FileChooser widget
fc = FileChooser('/users')

# === Change defaults and reset the dialog ===
# fc.default_path = './'
# fc.reset()

# === Restrict navigation to /Users ===
fc.sandbox_path = '/users'

# === Change hidden files ===
fc.show_hidden = False

# === Switch to folder-only mode ===
fc.show_only_dirs = False

# === Set multiple file filter patterns (uses https://docs.python.org/3/library/fnmatch.html) ===
fc.filter_pattern = ['*.csv', '*.mp4', '*.zip', '*.json']
# fc.filter_pattern = ['*.zip']

# === Change the title (use '' to hide) ===
fc.title = '<p>Select File (*.csv, *.txt, *.mp4, *.json, *.zip)<br>Use *.zip for multiple file uploads.<br></p><hr>'

# === Callback function ===
# def change_display_selected(chooser):
#     print(fc.selected_filename, end='\r')

# === Register callback function ===
# fc.register_callback(change_display_selected)
# ================ END FileChooser END ================
# ================ BTN CLICK UPLOAD ================
def on_button_clicked_upload(b):
#     print(fc.selected)
    fn = fc.selected_filename
    outputPath = ""
    if fn.endswith('.zip'):
        fn = fc.selected_filename.replace('.zip', '')
        outputPath = fn + " has been extracted and uploaded to:"
#         files = zf.ZipFile(fc.selected_path+"\\"+fc.selected_filename, 'r')
#         files.extractall('Data Folder\\'+fn)
#         files.close()

        with zf.ZipFile(fc.selected_path+"\\"+fc.selected_filename, 'r') as zipObj:
           # Get a list of all archived file names from the zip
            listOfFileNames = zipObj.namelist()
           # Iterate over the file names
            for fileName in listOfFileNames:
                # Check filename endswith csv
#                 if  fileName.endswith('.png') or fileName.endswith('.jpg') or fileName.endswith('.JPG'):
#                     # Extract a single file from zip
#                     zipObj.extract(fileName, 'Data_Folder\\Images')
#                     outputPath += "\n" + fileName + " >> Data_Folder\\Images"
                if fileName.endswith('.mp4'):
                    # Extract a single file from zip
                    zipObj.extract(fileName, 'Data_Folder\\Videos')
                    outputPath += "\n" + fileName + " >> Data_Folder\\Videos"
                elif fileName.endswith('.txt') or fileName.endswith('.csv') or fileName.endswith('.xlsx') or fileName.endswith('.json'):
                    zipObj.extract(fileName, 'Data_Folder\\Dataset')
                    outputPath += "\n" + fileName + " >> Data_Folder\\Dataset"
                else:
                    zipObj.extract(fileName, 'Data_Folder')
                    outputPath += "\n" + fileName + " >> Data_Folder"
    else:
        if fn.endswith('.png') or fn.endswith('.jpg') or fn.endswith('.JPG'):
            shutil.copyfile(fc.selected_path+"\\"+fc.selected_filename, 'Data_Folder\\Images\\'+fc.selected_filename)
            outputPath = fn + " has been uploaded to >> Data_Folder\\Images"
        elif fn.endswith('.mp4'):  
            shutil.copyfile(fc.selected_path+"\\"+fc.selected_filename, 'Data_Folder\\Videos\\'+fc.selected_filename)
            outputPath = fn + " has been uploaded to >> Data_Folder\\Videos"
        elif fn.endswith('.txt') or fn.endswith('.csv') or fn.endswith('.xlsx') or fn.endswith('.json'):
            shutil.copyfile(fc.selected_path+"\\"+fc.selected_filename, 'Data_Folder\\Dataset\\'+fc.selected_filename)
            outputPath = fn + " has been uploaded to >> Data_Folder\\Dataset"
        else:
            shutil.copyfile(fc.selected_path+"\\"+fc.selected_filename, 'Data_Folder\\'+fc.selected_filename)
            outputPath = fn + " has been uploaded to >> Data_Folder"
    with output:
#         print(fn + " >> " + outputPath + ".")
        print("***SUCCESS*** \n" + outputPath)
# ================ BTN CLICK UPLOAD END ================
# =============== CONFIRM BTN & OUTPUT ===============
confirmBtn = widgets.Button(
    description='Confirm Upload',
    button_style='info', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Confirm',
)
output = widgets.Output()
confirmBtn.on_click(on_button_clicked_upload)
# =============== CONFIRM BTN & OUTPUT END ===============
display(fc)
display(confirmBtn, output)

FileChooser(path='C:\Users', filename='', title='<p>Select File (*.csv, *.txt, *.mp4, *.json, *.zip)<br>Use *.…

Button(button_style='info', description='Confirm Upload', style=ButtonStyle(), tooltip='Confirm')

Output()

### Video Preview

In [57]:
from ipyfilechooser import FileChooser
from IPython.display import display, HTML, Video
import ipywidgets as widgets


output_test = widgets.Output()
# ========== FileChooser ==========
# Create and display a FileChooser widget
fc_test = FileChooser('./Data_Folder/Videos')
display(fc_test)

# Restrict navigation to /Users
fc_test.sandbox_path = './Data_Folder/Videos'

# Change hidden files
fc_test.show_hidden = False

# Switch to folder-only mode
fc_test.show_only_dirs = False

# Change the title (use '' to hide)
fc_test.title = '<b>Select input video</b>'

# Sample callback function
def change_display_selected(chooser):
    with output_test:        
        # Path to video
        video_path = './Data_Folder/Videos/'+fc_test.selected_filename

        # Video playback for preview
        display(HTML("""<video controls src=""" + video_path + """ type="video/mp4" width=100%><video/>"""))
    
# Register callback function
fc_test.register_callback(change_display_selected)
# ========== END FileChooser END ==========
display(output_test)

FileChooser(path='C:\Users\jaspz\Documents\GitHub\ict3104-team10-2022\Data_Folder\Videos', filename='', title=…

Output()

## Inference
This section in the notebook allows the user to perform inference using a pretrained HOI ML
model based on the TSU project.

In [68]:
import cv2
import pandas as pd
import ipywidgets as widgets
import zipfile as zf
import numpy as np
from IPython import display
from IPython.display import display, HTML, IFrame, clear_output
from ipyfilechooser import FileChooser
from moviepy.editor import VideoFileClip
import torch
from torch.autograd import Variable
import torch.nn.functional as F

# import importlib.util

# spec = importlib.util.spec_from_file_location("train", "./Training/train.py")
# train = importlib.util.module_from_spec(spec)
# spec.loader.exec_module(train)
import re
import csv
import json

# Load the label text file into a list
def load_labels():
    with open("./Testing/data/all_labels.txt") as file_in:
        lines = []
        for line in file_in:
            line = re.sub(r'\d+', '', line)
            line = line.strip()
            lines.append(line)
        return lines

# Turn ground truth/annotation to csv
def json_to_csv(vid_name, split_mode):
    
    # Load event label text file provided
    event_list = load_labels()
    
    # Remove extention from name
    video_name = vid_name[0:-4]
    
    # Open json file
    with open('./Testing/data/smarthome_'+split_mode+'_51.json') as json_file:
        data = json.load(json_file)
    
    # Header for csv
    fields = ['annotation', 'start_frame', 'end_frame'] 
    
    # For an item in JSON (is a string of name of video)
#     for action in data:
        
        # Load actions from data using action item
#         caption_data = data[str(action)]['actions']
    if data[video_name]['subset'] == 'testing':
        caption_data = data[video_name]['actions']

        # Open csv file object as variable
    #     with open('./Data_Folder/Annotations/annotation_'+str(action)+'.csv', 'w') as data_file:
        with open('./Data_Folder/Annotations/annotation_'+video_name+'.csv', 'w') as data_file:
            # Clean existing file if it is not empty
            data_file.truncate()
            csv_writer = csv.writer(data_file)
            csv_writer.writerow(fields)
            label_list = []
            # Loop to change the label number to the corresponding name from a list created from label txt file
            for action_list in caption_data:
                action_list[0] = event_list[int(action_list[0])]
                # Append updated list item into new list
                label_list.append(action_list)
            # Write new list to csv
            csv_writer.writerows(label_list)

            data_file.close()

            # Code to clear up empty rows
    #         df = pd.read_csv('./Data_Folder/Annotations/annotation_'+str(action)+'.csv')
    #         df.to_csv('./Data_Folder/Annotations/annotation_'+str(action)+'.csv', index=False)
            df = pd.read_csv('./Data_Folder/Annotations/annotation_'+video_name+'.csv')
            df.to_csv('./Data_Folder/Annotations/annotation_'+video_name+'.csv', index=False)

            return './Data_Folder/Annotations/annotation_'+video_name+'.csv'
    else:
        return ''

# Function to check dataframe of caption from testing model and ground truth
def check_df_caption_ground(df_caption, df_ground):
    cap = df_caption
    gro = df_ground
    
    # New dict to turn to dataframe at end
    new_cap = {'captions': [], 'match': []}
    
    # Get length of dataframe
    totalFrames = len(cap.index)
    
    totalFrames_g = len(gro.index)
    
    # Iterate while index is in range of length of dataframe
    for index in range(totalFrames):
        new_cap['captions'].append(cap['captions'][index])
        if index < totalFrames_g:
            # Check if the action matches
            if cap['captions'][index] == gro['annotation'][index]:
                new_cap['match'].append('T')
            else:
                new_cap['match'].append('F')
        else:
            new_cap['match'].append('F')
    
    # Return index and row in series
    return pd.DataFrame(new_cap).iterrows()
    

# Check if caption is for video
def check_csv_vidname(vid_name, caption):
    vid = vid_name[0:-4]
    cap = caption[8:-4]
    if vid == cap:
        return True
    else:
        return False
    
# To parse caption/annotation csv into dataframe
def parseCSV(caption, type):
    df = pd.read_csv(caption)
    if type == "c":
        header = 'captions'
    elif type == "g":
        header = 'annotation'
    captionsByFrame = {header: []}
    captions = ""
    totalFrames = len(df.index)
    initialStartFrame = 0
    
    # Iterate through to create new df with regards to start_frame, end_frame variable in csv
    for index in range(totalFrames):
        # Update startframe at start when move to next row of dataframe from csv
        newStartFrame = int(df['start_frame'][index]) - initialStartFrame
        for i in range(newStartFrame):
            captionsByFrame[header].append(captions)
        captions = str(df[header][index])
        initialStartFrame += newStartFrame
        if index == totalFrames-1:
            newStartFrame = int(df['end_frame'][index]) - initialStartFrame
            for j in range(newStartFrame):
                captionsByFrame[header].append(captions)

    return pd.DataFrame(captionsByFrame)

# Insert text into video
def output(frame):
    color = (255, 255, 255)
    value = ''
    try:
        # Assign this here as if call again will skip
        value = next(df_new)[1]
        # Check match column of df to change color to Green if T
        if str(value.match) == "T":
            color = (0, 255, 0)
        # Change color to red here
        else:
            color = (255, 0, 0)
        cv2.putText(frame, "Action: " + str(value.captions), position,
                    cv2.FONT_HERSHEY_SIMPLEX, 1, color, 2, cv2.LINE_AA)
    except StopIteration:
        pass
    
    return frame

def input(video, cap_csv, ground_csv, outputFilename):
    # Path to video
    video_path = video

    # Capture video
    cap = cv2.VideoCapture(video_path)

    # Get video height and width from captured video
    frame_width = int(cap.get(3))
    frame_height = int(cap.get(4))

    # Set position for text with offset of text size
    global position
    position = ((int((frame_width/4)-(268/2))), int(((frame_height/1.15)-(36/2))))

    video = VideoFileClip(video_path)

    # Path to caption
    caption = cap_csv
    
    # Path to ground truth
    ground = ground_csv

    global df_new
    # Store caption df
    df = parseCSV(caption, 'c')
    
    # Store ground truth df
    df_ground = parseCSV(ground, 'g')
    
    # Store caption df with match column
    df_new = check_df_caption_ground(df, df_ground)

    # Modifies the images of a clip by replacing the frame
    out_video = video.fl_image(output)

    # Write edited video to file
    out_video.write_videofile(outputFilename, audio=True)

    # Close video file clip
    out_video.close()

    return "Added captions to video"



### Select Model for Inference
The below code allows the user to select model(s) for inference only from the trained_models folder. After selection, the directory path and file name is used in the Run code for selected model.

In [69]:
# Install the following packages for HTML interactive widgets for Jupyter notebook
import ipywidgets as widgets
from IPython.display import display, IFrame, Video, clear_output
from ipyfilechooser import FileChooser

selectOutput = widgets.Output()

# (Split Setting Dropdown) for user to select
splitSettingInfer = widgets.Dropdown(
    options=['CS', 'CV'],
    value='CS',
    description='Split Setting Used on model:',
    disabled=False,
)
display(splitSettingInfer)


# ========== csvSelect ==========
# Create and display a FileChooser widget
csvSelect = FileChooser('./Data_Folder/Captions')
display(csvSelect)

# Restrict navigation to /Users
csvSelect.sandbox_path = './Data_Folder/Captions'

# Change hidden files
csvSelect.show_hidden = False

# Switch to folder-only mode
csvSelect.show_only_dirs = False

# Change the title (use '' to hide)
csvSelect.title = '<b>Select CSV file to use for predicted events</b>'

# Sample callback function
def change_display_selected_csvSelect(chooser):
    with selectOutput:
        print(csvSelect.selected_filename, end='\r')

# Register callback function
csvSelect.register_callback(change_display_selected_csvSelect)
# ========== END csvSelect END ==========



# ========== vidSelect ==========
# Create and display a FileChooser widget
vidSelect = FileChooser('./Data_Folder/Videos')
display(vidSelect)

# Restrict navigation to /Users
vidSelect.sandbox_path = './Data_Folder/Videos'

# Change hidden files
vidSelect.show_hidden = False

# Switch to folder-only mode
vidSelect.show_only_dirs = False

# Change the title (use '' to hide)
vidSelect.title = '<b>Select Video File</b>'

# Sample callback function
def change_display_selected_vidSelect(chooser):
    with selectOutput:
        print(vidSelect.selected_filename, end='\r')

# Register callback function
vidSelect.register_callback(change_display_selected_vidSelect)
# ========== END vidSelect END ==========



# ========== modelSelect ==========
# Create and display a FileChooser widget
modelSelect = FileChooser('./Trained_models/PDAN/')
display(modelSelect)

# Restrict navigation to /Users
modelSelect.sandbox_path = '../'

# Change hidden files
modelSelect.show_hidden = False

# Switch to folder-only mode
modelSelect.show_only_dirs = False

# Change the title (use '' to hide)
modelSelect.title = '<b>Select model file to use</b>'

# Sample callback function
def change_display_selected_modelSelect(chooser):
    with selectOutput:
        print(modelSelect.selected_filename, end='\r')

# Register callback function
modelSelect.register_callback(change_display_selected_modelSelect)
# ========== END modelSelect END ==========



trainButton = widgets.Button(description='Run',
                             layout=widgets.Layout(margin='20px 0px 0px 0px'))
btnOutput = widgets.Output()

def on_button_clicked(_):
    # "linking function with output"
    with btnOutput:
        # what happens when we press the button
        clear_output()
#         try:
        # Get annotate file path
        annotate_path = json_to_csv(vidSelect.selected_filename, splitSettingInfer.value)
        if check_csv_vidname(vidSelect.selected_filename, csvSelect.selected_filename):
            vidPath = vidSelect.selected_path + "/" + vidSelect.selected_filename
            csvPath = csvSelect.selected_path + "/" + csvSelect.selected_filename
            modelPath = modelSelect.selected_path + "/" + modelSelect.selected_filename
            %run -i ./Inferencing/inferencing.py -dataset TSU -mode rgb -split_setting $splitSettingInfer.value -model PDAN -test True -num_channel 512 -lr 0.0002 -kernelsize 2 -APtype map -epoch 1 -batch_size 1 -comp_info TSU_CS_RGB_PDAN -load_model $modelPath

    #         outputFilename = vidSelect.selected_filename[:-4] + "_output.mp4"
            outputFilename = "output.mp4"
            input(vidPath, csvPath, annotate_path, outputFilename)
            # print(vidSelect.selected_filename)
            display(HTML("""
                <video controls src="output.mp4" type="video/mp4" width=100%>
                <video/>
                """
            ))
        else:
#             print("Video name and csv do not match please try again")
#         except:
            print('Something has gone wrong please try again')
        
        
# linking button and function together using a button's method
trainButton.on_click(on_button_clicked)
# displaying button and its output together
widgets.VBox([trainButton, btnOutput])


Dropdown(description='Split Setting Used on model:', options=('CS', 'CV'), value='CS')

FileChooser(path='C:\Users\jaspz\Documents\GitHub\ict3104-team10-2022\Data_Folder\Captions', filename='', titl…

FileChooser(path='C:\Users\jaspz\Documents\GitHub\ict3104-team10-2022\Data_Folder\Videos', filename='', title=…

FileChooser(path='C:\Users\jaspz\Documents\GitHub\ict3104-team10-2022\Trained_models\PDAN', filename='', title…

### Select Videos for Feature Extraction
The below code allows the user to select video(s) for feature extraction only from the Videos folder located in the Feature_extraction folder. After selection, the directory path and file name is used in the Run code in the following cell.

In [30]:
from ipyfilechooser import FileChooser
from IPython.display import display
from IPython.display import IFrame
import ipywidgets as widgets
from IPython.display import Video

selected_videos = []

output_video_select = widgets.Output()
# ========== FileChooser ==========
# Create and display a FileChooser widget
fc_video_select = FileChooser('./Feature_extraction/Videos')
display(fc_video_select)

# Restrict navigation to /Users
fc_video_select.sandbox_path = './Feature_extraction/Videos'

# Change hidden files
fc_video_select.show_hidden = False

# Switch to folder-only mode
fc_video_select.show_only_dirs = False

# Change the title (use '' to hide)
fc_video_select.title = '<b>Select Video(s) for Feature extraction</b>'

# Sample callback function
def change_display_selected(chooser):
    with output_video_select:
        parsed_video = fc_video_select.selected_filename.split('.mp4')[0]
        if parsed_video not in selected_videos:
            selected_videos.append(parsed_video)
            print(fc_video_select.selected_filename + ' add to extraction list')
            
# Register callback function
fc_video_select.register_callback(change_display_selected)
# ========== END FileChooser END ==========
display(output_video_select)

FileChooser(path='C:\Users\jaspz\Documents\GitHub\ict3104-team10-2022\Feature_extraction\Videos', filename='',…

Output()

### Start Feature Extraction
The below code allows the user to start extracting feature(s) from the videos selected above. The features files are saved into the saved_TSU_features located inside the Features_extraction folder.

In [31]:
def btnClicked_runTraining(b):
    with outputRunTraining:
            print("Extracting features...")
    for video in selected_videos:
        fp = fc_video_select.selected_path+video+'.mp4'
        if fp:
            print("\n\nExtracting features from: " + video)
            %run -i ./Feature_extraction/Smarthome_extract_features_rewrite.py -window_size 16 -gpu 4 -split $video -mode rgb -root ./Feature_extraction/Videos/ -load_model ./Feature_extraction/TSU_16frame_weights_iter64000.pt -save_dir ./Feature_extraction/saved_TSU_features/
    print('\n\nExtracted features from selected video(s), see saved path for the respective feature file location')
        
runTrainingBtn = widgets.Button(
    description='Start extraction',
    button_style='info', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Confirm',
)

runTrainingBtn.on_click(btnClicked_runTraining)
outputRunTraining = widgets.Output()

display(runTrainingBtn, outputRunTraining)

Button(button_style='info', description='Start extraction', style=ButtonStyle(), tooltip='Confirm')

Output()

## Training
This section in the notebook allows the user to train a HOI ML model based on the TSU project.

### Neural Network
This section allows for modifications of the configurations in the Neural Network. 

In [47]:
#Imports
import ipywidgets as widgets
from IPython.display import clear_output

stagesDD = widgets.Dropdown(
    options=[1, 2, 3, 4, 5],
    value=1,
    description='Stages:',
    disabled=False,
)

layersDD = widgets.Dropdown(
    options=[1, 2, 3, 4, 5],
    value=1,
    description='Layers:',
    disabled=False,
)

classesBIT = widgets.BoundedIntText(
    value=157,
    min=1,
    max=160,
    step=1,
    description='Classes:',
    disabled=False
)

# Display widgets - Mode, Split_Setting, Model
row1_dropdowns = widgets.HBox([stagesDD, layersDD])
row2_dropdowns = widgets.HBox([classesBIT])

display(row1_dropdowns, row2_dropdowns)

### Select Model for Training
The below code allows the user to select model(s) for training only from the data_folder folder. After selection, the directory path and file name is used in the Run code for selected model.

In [66]:
# Install the following packages for HTML interactive widgets for Jupyter notebook
import ipywidgets as widgets
from IPython.display import display, IFrame, Video, clear_output
from ipyfilechooser import FileChooser

selectedModelPath = ""

outputSelectModel = widgets.Output()

# ========== FileChooser ==========
# Create and display a FileChooser widget
fcSelectModel = FileChooser('./Data_Folder')

# Restrict navigation to /Users
fcSelectModel.sandbox_path = './Data_Folder'

# Change hidden files
fcSelectModel.show_hidden = False

# Set multiple file filter patterns
fcSelectModel.filter_pattern = ['*']

# Switch to folder-only mode
fcSelectModel.show_only_dirs = False

# Change the title (use '' to hide)
fcSelectModel.title = '<b>Select File for Training</b>'

# callback function
def change_display_selected(chooser):
    selectedModelPath = fcSelectModel.selected_path+fcSelectModel.selected_filename
    with outputSelectModel:
        print("Directory path of the selected file >> " + selectedModelPath, end='\r')
        
#         print(fcSelectModel.selected_path+"\\"+fcSelectModel.selected_filename, end='\r')
#     runVid(fc2.selected_path+"\\"+fc2.selected_filename)
#         Video(fc2.selected_path+"\\"+fc2.selected_filename)
    
# Register callback function
fcSelectModel.register_callback(change_display_selected)
# ========== END FileChooser END ==========
        
display(outputSelectModel)
display(fcSelectModel)

Output()

FileChooser(path='C:\Users\jaspz\Documents\GitHub\ict3104-team10-2022\Data_Folder', filename='', title='<b>Sel…

### Run code for selected model
The code below is for user to train the model selected above.
- After selecting the epoch and batch size, the user just have to click run to start training the selected model.
- The user can also click clear to remove the outputs of the training. The user has to re-run the cell in order to run to start training the selected model.
- Upon running the run command, progress will be shown in the output cell below

In [49]:
#Imports
import ipywidgets as widgets
from IPython.display import clear_output
from ipywidgets import Label, Layout, TwoByTwoLayout, IntText, Button, Layout, jslink, IntText, IntSlider, GridspecLayout

trainBool = False

# Button to run Train.py
def btnClicked_runTraining(b):
    try:    
        selectedModelPath = fcSelectModel.selected_path+fcSelectModel.selected_filename
        with outputRunTraining:
            clear_output(wait=False)
            print("\nRunning the elected model >> " + selectedModelPath + " ...")

        # ---Running of Train.py Script---
    #     !python ./Training/train.py -dataset TSU -mode rgb -split_setting CS -model PDAN -train True -num_channel 512 -lr 0.0002 -kernelsize 2 -APtype map -epoch $epochDD.value -batch_size 1 -comp_info TSU_CS_RGB_PDAN -load_model ./Training/dataset/PDAN_TSU_RGB
        %run -i ./Training/train-modified.py -dataset TSU -mode $modeDD.value -split_setting $splitSettingDD.value -model $modelDD.value -train $trainTFCB.value -num_channel $num_channelDD.value -lr $lrBFT.value -kernelsize $kernelSizeDD.value -APtype $apTypeDD.value -epoch $epochBIT.value -batch_size $batchSizeDD.value -comp_info TSU_CS_RGB_PDAN -load_model $selectedModelPath -num_stages $stagesDD.value -num_layers $layersDD.value -num_classes $classesBIT.value 
    except:
        print("\nAn error has occured. Please check if a file has been selected for training.")
        
# Run Training Button Settings
runTrainingBtn = widgets.Button(
    description='Run Training Model',
    button_style='info', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Run training sequence',
    layout=Layout(
#     display='flex-end', 
#     align_items='center',
#     align_content='stretch',
    width='95%'
)
)

# (Mode Dropdown) for user to select
modeDD = widgets.Dropdown(
    options=['flow', 'skeleton', 'rgb'],
    value='rgb',
    disabled=False,
    description_tooltip="Only rgb is currently available."
)
# Mode type checking.
def on_change_modeDD(change):
    if change['type'] == 'change' and change['name'] == 'value':
        if change['new'] != 'rgb':
            runTrainingBtn.disabled = True
            with outputSelectedModel:
                print("'%s' Mode training is not yet available. Please use rgb." % change['new'], end='\r')
        else:
            runTrainingBtn.disabled = False
            with outputSelectedModel:
                clear_output(wait=False)
modeDD.observe(on_change_modeDD)

# (Split Setting Dropdown) for user to select
splitSettingDD = widgets.Dropdown(
    options=['CS', 'CV'],
    value='CS',
    disabled=False,
    description_tooltip="Split by CS or CV"
)

# (model Dropdown) for user to select
modelDD = widgets.Dropdown(
    options=['PDAN', 'Other'],
    value='PDAN',
    disabled=False,
    description_tooltip="Only PDAN is available for now"
)
# Model type checking.
def on_change_modelDD(change):
    if change['type'] == 'change' and change['name'] == 'value':
        if change['new'] == 'Other':
            runTrainingBtn.disabled = True
            with outputSelectedModel:
                print("'%s' Model training is not yet available. Please use PDAN." % change['new'], end='\r')
        else:
            runTrainingBtn.disabled = False
            with outputSelectedModel:
                clear_output(wait=False)
modelDD.observe(on_change_modelDD)

# (Train-T/F Checkbox) for users to select
trainTFCB = widgets.Checkbox(
    value=True,
    description='Train (T/F)',
    disabled=False,
    indent=True,
    display='flex',
    flex_flow='column',
    align_items='stretch',
    layout=Layout(
    align_items='center',
    justify_content='flex-end',
    tooltip="Set Train to True or False"
)
)

# (Num_Channel Dropdown) for user to select
num_channelDD = widgets.Dropdown(
    options=['256', '512', '1024'],
    value='512',
    disabled=False,
    display='flex',
    flex_flow='column',
    align_items='stretch',
    description_tooltip="Select the number of channels to be used"
)

# (lr Float Input) for user to enter (0-0.1)
lrBFT = widgets.BoundedFloatText(
    value=0.0002,
    min=0,
    max=0.1,
    step=0.0001,
    description='',
    disabled=False,
    display='flex',
    flex_flow='column',
    align_items='stretch',
    description_tooltip="Select the lr to be used"
)

# (kernelSize Dropdown) for user to select
kernelSizeDD = widgets.Dropdown(
    options=[1, 2, 3],
    value=1,
#     disabled=True,
    description_tooltip="Select the Kernel Size to be used"
)

# (APType Dropdown) for user to select
apTypeDD = widgets.Dropdown(
    options=['map', 'wap'],
    value='map',
#     description='APType:',
#     disabled=True,
    description_tooltip="Select the AP Type to be used"
)

# (Epoch Integer Input) for user to enter (1-140)
epochBIT = widgets.BoundedIntText(
    value=1,
    min=1,
    max=140,
    step=1,
#     description='Epoch:',
    disabled=False,
    display='flex',
    flex_flow='column',
    align_items='stretch',
    tooltip="Select number of epochs to run",
)

# (BatchSize Dropdown) for user to select
batchSizeDD = widgets.Dropdown(
    options=[1, 2, 3, 4, 5],
    value=1,
#     description='Batch Size:',
#     disabled=True,
    description_tooltip="Select the batch size",
#     layout=Layout(width="80%")
)

# Outputs 
outputRunTraining = widgets.Output()
outputSelectedModel = widgets.Output()
output = widgets.Output()

modehbox = widgets.HBox([Label('Mode: '), modeDD], layout=Layout(
    align_items='center',
    justify_content='flex-end'
))
splithbox = widgets.HBox([Label('Split Setting: '), splitSettingDD], layout=Layout(
    align_items='center',
    justify_content='flex-end'
))
modelhbox = widgets.HBox([Label('Model: '), modelDD], layout=Layout(
    align_items='center',
    justify_content='flex-end'
))
channelshbox = widgets.HBox([Label('Num_Channel: '), num_channelDD], layout=Layout(
    align_items='center',
    justify_content='flex-end'
))
lrhbox = widgets.HBox([Label('lr: '), lrBFT], layout=Layout(
    align_items='center',
    justify_content='flex-end'
))
kernelSizehbox = widgets.HBox([Label('Kernel Size: '), kernelSizeDD], layout=Layout(
    align_items='center',
    justify_content='flex-end'
))
apTypehbox = widgets.HBox([Label('AP Type: '), apTypeDD], layout=Layout(
    align_items='center',
    justify_content='flex-end'
))
epochhbox = widgets.HBox([Label('Epoch: '), epochBIT], layout=Layout(
    align_items='center',
    justify_content='flex-end'
))
batchSizehbox = widgets.HBox([Label('Batch Size: '), batchSizeDD], layout=Layout(
    align_items='center',
    justify_content='flex-end'
))


# Display widgets - Mode, Split_Setting, Model
# row1_widgets = widgets.HBox([modehbox, splithbox ], align='right', width='400px')
# row2_widgets = widgets.HBox([modelhbox, trainTFCB], align='right')
# row3_widgets = widgets.HBox([channelshbox, lrhbox],  align='right')
# row4_widgets = widgets.HBox([kernelSizehbox, apTypehbox], align='right')
# row5_widgets = widgets.HBox([epochBIT, batchSizehbox], align='right')
                         
# Display widgets - Train, num_channel, lr
# row2_widgets = widgets.HBox([trainTFCB, num_channelDD, lrBFT])
# Display widgets - kernelsize, APType, epoch, batchsize
# row3_widgets = widgets.HBox([kernelSizeDD, apTypeDD, epochBIT, batchSizeDD])

# Display rows of widgets
display(outputSelectedModel)
# Display Buttons
display(outputRunTrain5ing)

runTrainingBtn.on_click(btnClicked_runTraining)

# === Button to clear output
def on_buttonClear_clicked(b):
    clear_output(wait=False)
    
buttonClear = widgets.Button(description="Clear Output", layout=Layout(
#     display='flex-end', 
#     align_items='center',
#     align_content='stretch',
    width='95%'
))
buttonClear.on_click(on_buttonClear_clicked)

display(output)

# modeltxt = IntText(description='Model:', layout=Layout(width='auto', height='auto'))
# splittxt = IntText(description='Split Setting:', layout=Layout(width='auto', height='auto'))
# bottom_left_slider = IntSlider(description='Bottom left', layout=Layout(width='auto', height='auto'))
# bottom_right_slider = IntSlider(description='Bottom right', layout=Layout(width='auto', height='auto'))
# link_left = jslink((set1.top_left, 'value'), (set1.bottom_left, 'value'))
# link_right = jslink((set1.top_right, 'value'), (set1.bottom_right, 'value'))

grid = GridspecLayout(6, 2, height='300px')
grid[0, 0] = modehbox
grid[0, 1] = splithbox
grid[1, 0] = modelhbox
grid[1, 1] = trainTFCB
grid[2, 0] = channelshbox
grid[2, 1] = lrhbox
grid[3, 0] = kernelSizehbox
grid[3, 1] = apTypehbox
grid[4, 0] = epochhbox
grid[4, 1] = batchSizehbox
grid[5, 0] = buttonClear
grid[5, 1] = runTrainingBtn

grid

Output()

Output()

Output()

GridspecLayout(children=(HBox(children=(Label(value='Mode: '), Dropdown(index=2, options=('flow', 'skeleton', …

### Rename Trained Models

#### Select File to rename
The below code allows the user to select model(s) for training only from the /Trained_models/PDAN folder. After selection, the directory path and file name is used in the file rename function for selected model.

In [50]:
# Install the following packages for HTML interactive widgets for Jupyter notebook
import ipywidgets as widgets
from ipywidgets import Button
from IPython.display import display, HTML
from ipyfilechooser import FileChooser
import os

selectedModelPathToRename = ""

outputSelectModelToRename = widgets.Output()

# ========== FileChooser ==========
# Create and display a FileChooser widget
fcSelectModelToRename = FileChooser('./Trained_models/PDAN')

# Restrict navigation to /Trained_models/PDAN
fcSelectModelToRename.sandbox_path = './Trained_models/PDAN'

# Change hidden files
fcSelectModelToRename.show_hidden = False

# Set multiple file filter patterns
fcSelectModelToRename.filter_pattern = ['*']

# Switch to folder-only mode
fcSelectModelToRename.show_only_dirs = False

# Change the title (use '' to hide)
fcSelectModelToRename.title = '<b>Select File to rename</b>'

# callback function
def change_display_selected(chooser):
    selectedModelPathToRename = fcSelectModelToRename.selected_path+fcSelectModelToRename.selected_filename
    with outputSelectModelToRename:
        print("Directory path of the selected file >> " + selectedModelPathToRename,  end='\r')
    
# Register callback function
fcSelectModelToRename.register_callback(change_display_selected)
# ========== END FileChooser END ==========
display(outputSelectModelToRename)
display(fcSelectModelToRename)


Output()

FileChooser(path='C:\Users\jaspz\Documents\GitHub\ict3104-team10-2022\Trained_models\PDAN', filename='', title…

#### File rename function
The code below is for user to rename the trained model selected above.
- User will input the new model name in the textbox provided below
- The user will then click on the save changes button to rename the file
- Upon saving, there will be an output indicating the file has been renamed successfully.

In [51]:
import os

#Input for new filename
newFileName = widgets.Text(
    placeholder='Input new filename for selected file',
    disabled=False
)

#Save Changes Button
saveButton = widgets.Button(
    description='Save Changes',
    disabled=True,
    button_style='info',
    icon='check'
)

# Rename file when button is clicked and check if the filename exist.
def on_button_clicked(b):
    try:
        selectedModelCurrentFilePath = fcSelectModelToRename.selected_path+fcSelectModelToRename.selected_filename
        selectedModelNewFilePath = fcSelectModelToRename.selected_path+newFileName.value
        if os.path.exists(selectedModelNewFilePath):
            print("File name already exist!")
        else:
            os.rename( selectedModelCurrentFilePath, selectedModelNewFilePath)
            print("File renamed to "+ newFileName.value + " successfully!")
    except:
        print("\nAn error has occured. Please check if a file has been selected for renaming.")

# Disable save button if textbox is empty
def on_change_textbox(change):
    if len(newFileName.value) == 0:
        saveButton.disabled=True
    else:
        saveButton.disabled=False

newFileName.observe(on_change_textbox)
display(newFileName)
display(saveButton, output)
saveButton.on_click(on_button_clicked)

Text(value='', placeholder='Input new filename for selected file')

Button(button_style='info', description='Save Changes', disabled=True, icon='check', style=ButtonStyle())

Output()

## Testing
This section in the notebook allow the user to evaluate a trained model based on the TSU project.

### Set up WandB for data visualisation
- Create a wandb account on wandb.ai, create a new project in your account
- Run cell below to login, key in your API key which can be found here https://wandb.ai/authorize

In [70]:
import wandb
wandb.login()

True

#### Initialise WandB project
- key in your username and project name

In [71]:
import wandb
import ipywidgets as widgets
from colorama import Fore, Back

#Input username
username = widgets.Text(
    placeholder='Input account username',
    disabled=False
)

#Input project name
project = widgets.Text(
    placeholder='Input project name',
    disabled=False
)

#Intialise Project Button
intialiseProjectButton = widgets.Button(
    description='Intialise Project',
    button_style='info',
    icon='check',
    tooltip='Intialise Project',
)

ui = widgets.HBox([username, project])  
output = widgets.Output()
display(ui)
display(intialiseProjectButton, output)

# Rename file when button is clicked and check if the filename exist.
def on_save_clicked(b):
    with output:
        # Initialize WandB 
        wandb.init(entity=username.value, 
                   project= project.value,)
        print('\033[92m'+"\033[1m"+ "==============================================================="+ "\033[0m"+'\033[0m')
        print(Back.GREEN+"\033[1m"+ u'\u2713'+ "project initialised successfully"+ "\033[0m")


intialiseProjectButton.on_click(on_save_clicked)



Button(button_style='info', description='Intialise Project', icon='check', style=ButtonStyle(), tooltip='Intia…

Output()

### Select Model for testing
The below code allows the user to select pre-trained models only from the pre_trained_models folder. After selection, the directory path and file name is used in the Run code for selected model.

In [63]:
from ipyfilechooser import FileChooser
from IPython.display import display
from IPython.display import IFrame
import ipywidgets as widgets
from IPython.display import Video

output_test = widgets.Output()
# ========== FileChooser ==========
# Create and display a FileChooser widget
fc_test = FileChooser('./Pre_trained_models')
display(fc_test)

# Restrict navigation to /Users
fc_test.sandbox_path = './Pre_trained_models'

# Change hidden files
fc_test.show_hidden = False

# Switch to folder-only mode
fc_test.show_only_dirs = False

# Change the title (use '' to hide)
fc_test.title = '<b>Select File for Testing</b>'

# Sample callback function
def change_display_selected(chooser):
    with output_test:
#         print(fc2.selected_filename, end='\r')
        print(fc_test.selected_path+"\\"+fc_test.selected_filename + '\n\n', end='\r')
#     runVid(fc2.selected_path+"\\"+fc2.selected_filename)
#         Video(fc2.selected_path+"\\"+fc2.selected_filename)
    
# Register callback function
fc_test.register_callback(change_display_selected)
# ========== END FileChooser END ==========
display(output_test)

FileChooser(path='C:\Users\jaspz\Documents\GitHub\ict3104-team10-2022\Pre_trained_models', filename='', title=…

Output()

### Run code for selected model
The code below is for user to test the pre-trained model selected above, after selecting the epoch, the user just have to click run to start testing the selected model. 
- Results of the testing are then saved into CSV into the results folder under Testing.
- Graphs are generated in WandB

In [64]:
import ipywidgets as widgets
from IPython.display import display

def btnClicked_runTraining(b):
#     print(epochDD.value)
    with outputRunTraining:
        try:
            print("Testing model " + fc_test.selected_filename)
            fp = fc_test.selected_path+fc_test.selected_filename
            print("Running...")
            #fp = fc2.selected_path+fc2.selected_filename
            %run -i ./Testing/test.py -dataset TSU -mode rgb -split_setting CS -model PDAN -test True -num_channel 512 -lr 0.0002 -kernelsize 2 -APtype map -epoch 1 -batch_size 1 -comp_info TSU_CS_RGB_PDAN -load_model $fp
        except:
            print("\nAn error has occured. Please check if a file has been selected for training.")
    
runTrainingBtn = widgets.Button(
    description='Run Testing Model',
    button_style='info', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Confirm',
)

runTrainingBtn.on_click(btnClicked_runTraining)
outputRunTraining = widgets.Output()

display(runTrainingBtn, outputRunTraining)

Button(button_style='info', description='Run Testing Model', style=ButtonStyle(), tooltip='Confirm')

Output()

#### View WandB project workspace

In [72]:
import wandb
import ipywidgets as widgets

%wandb {username.value}/{project.value}


# STEP Pipeline
This section in the notebook is delicated for the STEP pipline.

## To be added for STEP pipeline stuff..